# Section B: Practical questions with applied multiple choice

## General Rules:
- This is an open book examination.
- Students may make use of a calculator.
- This is an online examination where you will access a computer; however you may not communicate with other students in any form.
- Headphone are prohibited.
- The use of AI (chatGPT etc.) is prohibited.
- All cell phones are to be switched off for the duration of the exam.
- The invigilator will not assist you with the explanation of questions.
- Students are prohibited from conversing in any manner with other students.

## My Name and Surname

Name = 
</br>
Surname =  

### Part 1: SQL Queries  
You are provided with a pre-populated SQLite database named `airbnb.db`. Download [here](https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata) if you haven't already. Your task is to explore this database and write a series of SQL queries to perform the tasks detailed below. Queries should be optimised to run within 20 seconds or less.

The tables and columns included in the `airbnb.db` are:

- `listings`: `listing_id`, `host_id`, `listing_name`, `neighbourhood`, `room_type`, `price`, `minimum_nights`, `number_of_reviews`, `last_review`, `reviews_per_month`, `calculated_host_listings_count`, `availability_365`  
- `hosts`: `host_id`, `host_name`, `host_since`, `host_location`, `host_response_time`, `host_response_rate`, `host_is_superhost`  
- `reviews`: `review_id`, `listing_id`, `reviewer_id`, `review_date`, `comments`  
- `reviewers`: `reviewer_id`, `reviewer_name`  
- `calendar`: `listing_id`, `date`, `available`, `price`  
- `neighbourhoods`: `neighbourhood`, `borough`  
- `amenities`: `listing_id`, `amenity_name`

In [ ]:
import os
import json
import random
import sqlite3
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

# Load the Airbnb dataset from the local path
file_path = r"C:\Users\User\Desktop\Predict\Airbnb_Open_Data.csv"

try:
    airbnb_df = pd.read_csv(file_path)
    print("CSV file loaded successfully.")
    print("First 5 rows of the dataset:")
    display(airbnb_df.head())
except FileNotFoundError as e:
    print("Failed to load CSV file:", e)



In [ ]:
# List all tables in the database
        tables = pd.read_sql('''SELECT name FROM sqlite_master WHERE type='table';''', conn)
        print("Tables in the database:")
        display(tables)

#### SQL Query 1: Top 10 Neighbourhoods by Average Listing Price

#### This query helps identify the most expensive areas to stay in San Francisco.
#### It calculates the average price of listings per neighbourhood, excluding free or zero-priced listings,
#### and returns the top 10 neighbourhoods with the highest average prices.




In [ ]:
query1 = '''
SELECT 
    neighbourhood, 
    AVG(price) AS average_price
FROM 
    listings
WHERE 
    price > 0
GROUP BY 
    neighbourhood
ORDER BY 
    average_price DESC
LIMIT 10;
'''